In [ ]:
%matplotlib widget
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from matplotlib.widgets import RectangleSelector

from wandas.core import ChannelFrame

In [ ]:
# ダミーデータの生成（省略可）
np.random.seed(0)
data = np.random.randn(100, 10000)
sampling_rate = 1000
labels = [f"Signal {i}" for i in range(1000)]
signals = ChannelFrame.from_ndarray(
    array=data, sampling_rate=sampling_rate, labels=labels
)
max_values = np.array([signals[i].data.max() for i in range(len(signals))])

In [ ]:
class IndexTable(widgets.VBox):
    def __init__(self):
        super().__init__()
        self.indices = []
        self.items_per_page = 10
        self.current_page = 0

        # 全ページにわたる選択状態を保持するセット
        self.selected_indices_set = set()

        self.selected_count_label = widgets.HTML(
            value="Selected: <span style='border:2px solid red; padding:2px;'>0</span>",
            layout=widgets.Layout(margin="0px 0px 10px 0px"),
        )

        # ページ送りボタン
        self.prev_button = widgets.Button(description="← 戻る", disabled=True)
        self.next_button = widgets.Button(description="進む →", disabled=True)
        self.prev_button.on_click(self.on_prev_click)
        self.next_button.on_click(self.on_next_click)

        # すべて選択 / すべて解除 ボタン
        self.select_all_button = widgets.Button(description="すべて選択")
        self.clear_all_button = widgets.Button(description="すべて解除")
        self.select_all_button.on_click(self.on_select_all_click)
        self.clear_all_button.on_click(self.on_clear_all_click)

        # ソート用UI
        self.sort_column_dropdown = widgets.Dropdown(
            options=["Index", "Max Value"], value="Index", description="列名:"
        )
        self.sort_asc_button = widgets.Button(description="昇順")
        self.sort_desc_button = widgets.Button(description="降順")
        self.sort_asc_button.on_click(self.on_sort_asc_click)
        self.sort_desc_button.on_click(self.on_sort_desc_click)

        self.button_box = widgets.HBox(
            [
                self.prev_button,
                self.next_button,
                self.select_all_button,
                self.clear_all_button,
            ]
        )
        self.sort_box = widgets.HBox(
            [self.sort_column_dropdown, self.sort_asc_button, self.sort_desc_button]
        )

        self.table_container = widgets.Output()
        self.table_container.layout.height = "600px"
        self.table_container.layout.overflow_y = "auto"
        self.table_container.layout.border = "solid 1px #ccc"
        self.table_container.layout.padding = "0px"

        self.detail_output = widgets.Output()
        self.detail_output.layout.height = "400px"
        self.detail_output.layout.overflow_y = "auto"
        self.detail_output.layout.border = "solid 1px #ccc"
        self.detail_output.layout.padding = "10px"

        self.children = [
            self.selected_count_label,
            self.button_box,
            self.sort_box,
            self.table_container,
            # self.detail_output
        ]

    def update_table(self, indices):
        self.indices = indices
        self.current_page = 0
        self.show_page()

    def show_page(self):
        self.checkboxes = []

        with self.table_container:
            self.table_container.clear_output()

            total_items = len(self.indices)
            total_pages = max((total_items - 1) // self.items_per_page + 1, 1)

            start = self.current_page * self.items_per_page
            end = start + self.items_per_page
            page_indices = self.indices[start:end]

            header_style = (
                "font-weight:bold; background:#f7f7f7; "
                "border-bottom:1px solid #ccc; padding:4px; text-align:center;"
            )
            cell_style = (
                "border-bottom:1px solid #eee; padding:4px; "
                "text-align:center; white-space:nowrap;"
            )

            children = []
            header_select = widgets.HTML(
                value=f"<div style='{header_style}'>Select</div>"
            )
            header_index = widgets.HTML(
                value=f"<div style='{header_style}'>Index</div>"
            )
            header_value = widgets.HTML(
                value=f"<div style='{header_style}'>Max Value</div>"
            )
            children.extend([header_select, header_index, header_value])

            if len(page_indices) > 0:
                for idx in page_indices:
                    selected = idx in self.selected_indices_set
                    cb = widgets.Checkbox(value=selected, indent=False)
                    cb.observe(self.on_checkbox_change, "value")
                    self.checkboxes.append((cb, idx))

                    cb_box = widgets.HBox(
                        [cb],
                        layout=widgets.Layout(
                            justify_content="center",
                            border_bottom="1px solid #eee",
                            padding="4px",
                        ),
                    )
                    index_html = widgets.HTML(
                        value=f"<div style='{cell_style}'>{idx}</div>"
                    )
                    val_html = widgets.HTML(
                        value=f"<div style='{cell_style}'>{max_values[idx]:.2f}</div>"
                    )

                    children.extend([cb_box, index_html, val_html])
                n_rows = 1 + len(page_indices)
            else:
                empty_select = widgets.HTML(value=f"<div style='{cell_style}'></div>")
                empty_index = widgets.HTML(
                    value=f"<div style='{cell_style}'>No indices selected.</div>"
                )
                empty_value = widgets.HTML(value=f"<div style='{cell_style}'></div>")
                children.extend([empty_select, empty_index, empty_value])
                n_rows = 2

            grid = widgets.GridBox(
                children=children,
                layout=widgets.Layout(
                    width="auto",
                    grid_template_columns="60px 50px 80px",
                    grid_template_rows=f"repeat({n_rows}, auto)",
                    border="none",
                    grid_gap="0px",
                    background_color="white",
                    align_items="stretch",
                ),
            )
            display(grid)

        self.prev_button.disabled = self.current_page == 0
        self.next_button.disabled = self.current_page == total_pages - 1

        self.update_detail()

    def set_selected_count(self, count):
        self.selected_count_label.value = (
            f"Selected: <span style='border:2px solid red; padding:2px;'>{count}</span>"
        )

    def on_checkbox_change(self, change):
        for cb, idx in self.checkboxes:
            if cb is change.owner:
                if cb.value:
                    self.selected_indices_set.add(idx)
                else:
                    self.selected_indices_set.discard(idx)
                break

        self.set_selected_count(len(self.selected_indices_set))
        self.update_detail()

    def update_detail(self):
        self.detail_output.clear_output()
        selected_list = sorted(self.selected_indices_set)
        box = widgets.VBox([signals[idx].describe() for idx in selected_list])
        with self.detail_output:
            display(box)

    def on_prev_click(self, b):
        if self.current_page > 0:
            self.current_page -= 1
            self.show_page()

    def on_next_click(self, b):
        total_items = len(self.indices)
        total_pages = max((total_items - 1) // self.items_per_page + 1, 1)
        if self.current_page < total_pages - 1:
            self.current_page += 1
            self.show_page()

    def on_select_all_click(self, b):
        for cb, idx in self.checkboxes:
            cb.value = True

    def on_clear_all_click(self, b):
        for cb, idx in self.checkboxes:
            cb.value = False

    def on_sort_asc_click(self, b):
        self.sort_indices(ascending=True)
        self.show_page()

    def on_sort_desc_click(self, b):
        self.sort_indices(ascending=False)
        self.show_page()

    def sort_indices(self, ascending=True):
        col = self.sort_column_dropdown.value
        if col == "Index":
            self.indices = sorted(self.indices, reverse=not ascending)
        elif col == "Max Value":
            self.indices = sorted(
                self.indices, key=lambda x: max_values[x].item(), reverse=not ascending
            )


index_table = IndexTable()


def onselect(eclick, erelease):
    x1, _ = eclick.xdata, eclick.ydata
    x2, _ = erelease.xdata, erelease.ydata

    selected_indices = np.where((max_values >= x1) & (max_values <= x2))[0]

    index_table.selected_indices_set.clear()
    index_table.set_selected_count(0)

    # テーブルを更新
    index_table.update_table(selected_indices)


# それぞれの Figure を widgets.Output に入れて表示
hist_output = widgets.Output()
# -------------------------------------------
# ポイント：fig作成後すぐに close して自動描画を抑制する
# -------------------------------------------


# ヒストグラムを初期描画
def plot_histogram():
    hist_ax.clear()
    hist_ax.hist(max_values, bins=50)
    hist_ax.set_xlabel("Max Value")
    hist_ax.set_ylabel("Frequency")
    hist_ax.set_title("Histogram of Max Values of Signals")
    # RectangleSelector をヒストグラムに紐づける
    global RS
    RS = RectangleSelector(
        hist_ax,
        onselect,
        useblit=True,
        minspanx=5,
        minspany=5,
        spancoords="pixels",
        interactive=True,
    )
    # hist_fig.canvas.draw()  # 必要に応じて呼び出す（なくてもOK）


plt.ioff()
hist_fig, hist_ax = plt.subplots(figsize=(10, 4))
plot_histogram()
plt.ion()
with hist_output:
    hist_fig.show()

# 全体レイアウト
layout = widgets.VBox(
    [widgets.HBox([hist_output, index_table]), index_table.detail_output]
)
display(layout)

/usr/local/lib/python3.10/site-packages/traitlets/traitlets.py:1385: DeprecationWarning: Passing unrecognized arguments to super(Toolbar).__init__().
NavigationToolbar2WebAgg.__init__() missing 1 required positional argument: 'canvas'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  warn(
